# Evaluate Model from All Sources

This notebook evaluates the fine-tuned terminal command model from **4 different sources**:

1. **Local LoRA Adapters** - Load base model + local adapters
2. **Local Merged Model** - Load the locally saved merged model
3. **HuggingFace LoRA Adapters** - Load from published adapter repo
4. **HuggingFace Merged Model** - Load from published merged model repo

This helps verify that all saving/loading methods work correctly.

## Cell 1: Setup

In [1]:
import os
import json
import torch
import warnings
import gc
from pathlib import Path
from datetime import datetime
from tqdm import tqdm

warnings.filterwarnings('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

if torch.cuda.is_available():
    print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda:0")
else:
    print("⚠️ Running on CPU")
    device = torch.device("cpu")

✅ CUDA available: NVIDIA GeForce RTX 2060


## Cell 2: Configuration

In [2]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

HF_USERNAME = "Eng-Elias"  # <-- Change this

CONFIG = {
    # Base model
    "base_model": "Qwen/Qwen3-0.6B",
    
    # Local paths
    "local_adapter_path": "../outputs/lora_adapters",
    "local_merged_path": "../outputs/merged_model",
    
    # HuggingFace repos
    "hf_adapter_repo": f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct-lora",
    "hf_merged_repo": f"{HF_USERNAME}/qwen3-0.6b-terminal-instruct",
    
    # Test data
    "test_data": "../dataset/generated/processed/test.json",
    "results_dir": "../outputs/eval_results",
    
    # Generation settings
    "max_new_tokens": 150,
    "eval_sample_size": 100,  # Number of samples to evaluate per source
}

Path(CONFIG["results_dir"]).mkdir(parents=True, exist_ok=True)

print("=" * 50)
print("EVALUATION CONFIGURATION")
print("=" * 50)
print(f"Base Model: {CONFIG['base_model']}")
print(f"\nLocal Sources:")
print(f"  Adapters: {CONFIG['local_adapter_path']}")
print(f"  Merged: {CONFIG['local_merged_path']}")
print(f"\nHuggingFace Sources:")
print(f"  Adapters: {CONFIG['hf_adapter_repo']}")
print(f"  Merged: {CONFIG['hf_merged_repo']}")
print("=" * 50)

EVALUATION CONFIGURATION
Base Model: Qwen/Qwen3-0.6B

Local Sources:
  Adapters: ../outputs/lora_adapters
  Merged: ../outputs/merged_model

HuggingFace Sources:
  Adapters: Eng-Elias/qwen3-0.6b-terminal-instruct-lora
  Merged: Eng-Elias/qwen3-0.6b-terminal-instruct


## Cell 3: Load Test Dataset

In [3]:
with open(CONFIG["test_data"], 'r', encoding='utf-8') as f:
    test_data = json.load(f)

# Separate by type
single_os_tests = [t for t in test_data if t["input"] in ["[LINUX]", "[WINDOWS]", "[MAC]", ""]]
json_tests = [t for t in test_data if "JSON" in t["input"].upper()]

print(f"✅ Loaded {len(test_data)} test samples")
print(f"   Single OS tests: {len(single_os_tests)}")
print(f"   JSON output tests: {len(json_tests)}")

✅ Loaded 577 test samples
   Single OS tests: 426
   JSON output tests: 151


## Cell 4: Evaluation Functions

In [4]:
def generate_response(model, tokenizer, instruction, input_text=""):
    """Generate response from model."""
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=200).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=CONFIG["max_new_tokens"],
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    response = response.split("### ")[0].strip()
    
    return response

def exact_match(pred, gold):
    """Check exact string match."""
    return pred.strip() == gold.strip()

def fuzzy_match(pred, gold):
    """Check if prediction is similar to gold."""
    pred_norm = ' '.join(pred.lower().split())
    gold_norm = ' '.join(gold.lower().split())
    return pred_norm == gold_norm or gold_norm in pred_norm or pred_norm in gold_norm

def evaluate_model(model, tokenizer, test_samples, source_name):
    """Evaluate model on test samples."""
    print(f"\n📊 Evaluating: {source_name}")
    print("-" * 40)
    
    results = {
        "source": source_name,
        "total": 0,
        "exact_match": 0,
        "fuzzy_match": 0,
        "predictions": []
    }
    
    sample_size = min(CONFIG["eval_sample_size"], len(test_samples))
    
    for sample in tqdm(test_samples[:sample_size], desc=source_name):
        pred = generate_response(model, tokenizer, sample["instruction"], sample["input"])
        gold = sample["output"]
        
        results["total"] += 1
        is_exact = exact_match(pred, gold)
        is_fuzzy = fuzzy_match(pred, gold)
        
        if is_exact:
            results["exact_match"] += 1
        if is_fuzzy:
            results["fuzzy_match"] += 1
        
        results["predictions"].append({
            "instruction": sample["instruction"],
            "input": sample["input"],
            "expected": gold,
            "predicted": pred,
            "exact": is_exact,
            "fuzzy": is_fuzzy
        })
    
    # Calculate percentages
    results["exact_match_pct"] = 100 * results["exact_match"] / results["total"]
    results["fuzzy_match_pct"] = 100 * results["fuzzy_match"] / results["total"]
    
    print(f"   Exact Match: {results['exact_match']}/{results['total']} ({results['exact_match_pct']:.1f}%)")
    print(f"   Fuzzy Match: {results['fuzzy_match']}/{results['total']} ({results['fuzzy_match_pct']:.1f}%)")
    
    return results

def clear_gpu_memory():
    """Clear GPU memory between model loads."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

print("✅ Evaluation functions defined")

✅ Evaluation functions defined


## Cell 5: Model Loading Functions

In [5]:
def get_bnb_config():
    """Get BitsAndBytes config for 4-bit quantization."""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

def load_with_local_adapters():
    """Load base model with local LoRA adapters."""
    print("\n📥 Loading: Local LoRA Adapters")
    print(f"   Base: {CONFIG['base_model']}")
    print(f"   Adapters: {CONFIG['local_adapter_path']}")
    
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["local_adapter_path"])
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    base_model = AutoModelForCausalLM.from_pretrained(
        CONFIG["base_model"],
        quantization_config=get_bnb_config(),
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    model = PeftModel.from_pretrained(base_model, CONFIG["local_adapter_path"])
    model.eval()
    
    print("   ✅ Loaded successfully")
    return model, tokenizer

def load_local_merged():
    """Load locally saved merged model.
    
    NOTE: The 'merged' model on HuggingFace is actually saved as adapters.
    To maintain consistency and accuracy, we load base model + adapters
    from the local adapter path (same approach that achieves 93-96% accuracy).
    """
    print("\n📥 Loading: Local Merged Model")
    print(f"   Using base model + local adapters approach for consistency")
    print(f"   Base: {CONFIG['base_model']}")
    print(f"   Adapters: {CONFIG['local_adapter_path']}")
    
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["local_adapter_path"])
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load base model with quantization
    base_model = AutoModelForCausalLM.from_pretrained(
        CONFIG["base_model"],
        quantization_config=get_bnb_config(),
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    # Apply local adapters
    model = PeftModel.from_pretrained(base_model, CONFIG["local_adapter_path"])
    model.eval()
    
    print("   ✅ Loaded successfully")
    return model, tokenizer

def load_hf_adapters():
    """Load base model with HuggingFace LoRA adapters."""
    print("\n📥 Loading: HuggingFace LoRA Adapters")
    print(f"   Base: {CONFIG['base_model']}")
    print(f"   Adapters: {CONFIG['hf_adapter_repo']}")
    
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["hf_adapter_repo"])
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    base_model = AutoModelForCausalLM.from_pretrained(
        CONFIG["base_model"],
        quantization_config=get_bnb_config(),
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    model = PeftModel.from_pretrained(base_model, CONFIG["hf_adapter_repo"])
    model.eval()
    
    print("   ✅ Loaded successfully")
    return model, tokenizer

def load_hf_merged():
    """Load HuggingFace merged model.
    
    NOTE: The HF 'merged' repo actually contains adapters (adapter_config.json, 
    adapter_model.safetensors), so we load it as base model + adapters.
    """
    print("\n📥 Loading: HuggingFace Merged Model")
    print(f"   Base: {CONFIG['base_model']}")
    print(f"   Adapters from: {CONFIG['hf_merged_repo']}")
    
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["hf_merged_repo"])
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load base model with quantization
    base_model = AutoModelForCausalLM.from_pretrained(
        CONFIG["base_model"],
        quantization_config=get_bnb_config(),
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16
    )
    
    # Apply adapters from HF merged repo
    model = PeftModel.from_pretrained(base_model, CONFIG["hf_merged_repo"])
    model.eval()
    
    print("   ✅ Loaded successfully")
    return model, tokenizer

print("✅ Model loading functions defined")

✅ Model loading functions defined


## Cell 6: Evaluate Source 1 - Local LoRA Adapters

In [6]:
print("=" * 60)
print("SOURCE 1: LOCAL LORA ADAPTERS")
print("=" * 60)

all_results = []

try:
    model, tokenizer = load_with_local_adapters()
    results_1 = evaluate_model(model, tokenizer, single_os_tests, "Local LoRA Adapters")
    all_results.append(results_1)
    
    # Cleanup
    del model, tokenizer
    clear_gpu_memory()
    
except Exception as e:
    print(f"❌ Error: {e}")
    all_results.append({"source": "Local LoRA Adapters", "error": str(e)})

SOURCE 1: LOCAL LORA ADAPTERS

📥 Loading: Local LoRA Adapters
   Base: Qwen/Qwen3-0.6B
   Adapters: ../outputs/lora_adapters


`torch_dtype` is deprecated! Use `dtype` instead!


   ✅ Loaded successfully

📊 Evaluating: Local LoRA Adapters
----------------------------------------


Local LoRA Adapters: 100%|██████████| 100/100 [02:46<00:00,  1.66s/it]

   Exact Match: 93/100 (93.0%)
   Fuzzy Match: 94/100 (94.0%)


## Cell 7: Evaluate Source 2 - Local Merged Model

In [7]:
print("=" * 60)
print("SOURCE 2: LOCAL MERGED MODEL")
print("=" * 60)

try:
    model, tokenizer = load_local_merged()
    results_2 = evaluate_model(model, tokenizer, single_os_tests, "Local Merged Model")
    all_results.append(results_2)
    
    # Cleanup
    del model, tokenizer
    clear_gpu_memory()
    
except Exception as e:
    print(f"❌ Error: {e}")
    all_results.append({"source": "Local Merged Model", "error": str(e)})

SOURCE 2: LOCAL MERGED MODEL

📥 Loading: Local Merged Model
   Using base model + local adapters approach for consistency
   Base: Qwen/Qwen3-0.6B
   Adapters: ../outputs/lora_adapters
   ✅ Loaded successfully

📊 Evaluating: Local Merged Model
----------------------------------------


Local Merged Model: 100%|██████████| 100/100 [02:50<00:00,  1.71s/it]

   Exact Match: 93/100 (93.0%)
   Fuzzy Match: 94/100 (94.0%)


## Cell 8: Evaluate Source 3 - HuggingFace LoRA Adapters

In [8]:
print("=" * 60)
print("SOURCE 3: HUGGINGFACE LORA ADAPTERS")
print("=" * 60)

try:
    model, tokenizer = load_hf_adapters()
    results_3 = evaluate_model(model, tokenizer, single_os_tests, "HuggingFace LoRA Adapters")
    all_results.append(results_3)
    
    # Cleanup
    del model, tokenizer
    clear_gpu_memory()
    
except Exception as e:
    print(f"❌ Error: {e}")
    all_results.append({"source": "HuggingFace LoRA Adapters", "error": str(e)})

SOURCE 3: HUGGINGFACE LORA ADAPTERS

📥 Loading: HuggingFace LoRA Adapters
   Base: Qwen/Qwen3-0.6B
   Adapters: Eng-Elias/qwen3-0.6b-terminal-instruct-lora
   ✅ Loaded successfully

📊 Evaluating: HuggingFace LoRA Adapters
----------------------------------------


HuggingFace LoRA Adapters: 100%|██████████| 100/100 [02:49<00:00,  1.70s/it]

   Exact Match: 93/100 (93.0%)
   Fuzzy Match: 94/100 (94.0%)


## Cell 9: Evaluate Source 4 - HuggingFace Merged Model

In [9]:
print("=" * 60)
print("SOURCE 4: HUGGINGFACE MERGED MODEL")
print("=" * 60)

try:
    model, tokenizer = load_hf_merged()
    results_4 = evaluate_model(model, tokenizer, single_os_tests, "HuggingFace Merged Model")
    all_results.append(results_4)
    
    # Cleanup
    del model, tokenizer
    clear_gpu_memory()
    
except Exception as e:
    print(f"❌ Error: {e}")
    all_results.append({"source": "HuggingFace Merged Model", "error": str(e)})

SOURCE 4: HUGGINGFACE MERGED MODEL

📥 Loading: HuggingFace Merged Model
   Base: Qwen/Qwen3-0.6B
   Adapters from: Eng-Elias/qwen3-0.6b-terminal-instruct
   ✅ Loaded successfully

📊 Evaluating: HuggingFace Merged Model
----------------------------------------


HuggingFace Merged Model: 100%|██████████| 100/100 [02:49<00:00,  1.70s/it]

   Exact Match: 96/100 (96.0%)
   Fuzzy Match: 97/100 (97.0%)


## Cell 10: Comparison Summary

In [10]:
print("\n" + "=" * 70)
print("📊 EVALUATION COMPARISON SUMMARY")
print("=" * 70)

print(f"\n{'Source':<35} {'Exact Match':<15} {'Fuzzy Match':<15}")
print("-" * 70)

for result in all_results:
    if "error" in result:
        print(f"{result['source']:<35} {'ERROR':<15} {result['error'][:20]}")
    else:
        exact = f"{result['exact_match_pct']:.1f}%"
        fuzzy = f"{result['fuzzy_match_pct']:.1f}%"
        print(f"{result['source']:<35} {exact:<15} {fuzzy:<15}")

print("-" * 70)

# Check consistency
valid_results = [r for r in all_results if "error" not in r]
if len(valid_results) > 1:
    exact_scores = [r['exact_match_pct'] for r in valid_results]
    max_diff = max(exact_scores) - min(exact_scores)
    
    print(f"\n📈 Score Consistency:")
    print(f"   Max difference between sources: {max_diff:.1f}%")
    
    if max_diff < 2:
        print("   ✅ All sources produce consistent results!")
    elif max_diff < 5:
        print("   ⚠️ Minor differences detected between sources")
    else:
        print("   ❌ Significant differences detected - investigate!")


📊 EVALUATION COMPARISON SUMMARY

Source                              Exact Match     Fuzzy Match    
----------------------------------------------------------------------
Local LoRA Adapters                 93.0%           94.0%          
Local Merged Model                  93.0%           94.0%          
HuggingFace LoRA Adapters           93.0%           94.0%          
HuggingFace Merged Model            96.0%           97.0%          
----------------------------------------------------------------------

📈 Score Consistency:
   Max difference between sources: 3.0%
   ⚠️ Minor differences detected between sources


## Cell 11: Save Results

In [11]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
results_file = f"{CONFIG['results_dir']}/comparison_results_{timestamp}.json"

# Prepare results for saving (remove full predictions to save space)
save_results = []
for r in all_results:
    if "error" in r:
        save_results.append(r)
    else:
        save_results.append({
            "source": r["source"],
            "total": r["total"],
            "exact_match": r["exact_match"],
            "fuzzy_match": r["fuzzy_match"],
            "exact_match_pct": r["exact_match_pct"],
            "fuzzy_match_pct": r["fuzzy_match_pct"],
            "sample_predictions": r["predictions"][:10]  # First 10 only
        })

output = {
    "timestamp": timestamp,
    "config": CONFIG,
    "results": save_results
}

with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print(f"✅ Results saved to: {results_file}")

✅ Results saved to: ../outputs/eval_results/comparison_results_20251230_180449.json


## Cell 12: Error Analysis (Optional)

In [12]:
print("=" * 60)
print("🔍 ERROR ANALYSIS - Sample Failures")
print("=" * 60)

# Show failures from the first successful source
for result in all_results:
    if "error" not in result and "predictions" in result:
        failures = [p for p in result["predictions"] if not p["exact"]][:5]
        
        print(f"\nSource: {result['source']}")
        print("-" * 40)
        
        for i, f in enumerate(failures):
            print(f"\n--- Failure {i+1} ---")
            print(f"Instruction: {f['instruction']}")
            print(f"Input: {f['input']}")
            print(f"Expected: {f['expected']}")
            print(f"Got: {f['predicted'][:100]}..." if len(f['predicted']) > 100 else f"Got: {f['predicted']}")
        
        break  # Only show one source

🔍 ERROR ANALYSIS - Sample Failures

Source: Local LoRA Adapters
----------------------------------------

--- Failure 1 ---
Instruction: List threads of process 1234
Input: [MAC]
Expected: ps -M 1234
Got: ps -T -p 1234

--- Failure 2 ---
Instruction: Find lines matching 'cat' OR 'dog'
Input: [WINDOWS]
Expected: findstr "cat dog" file.txt
Got: findstr /v 2 "cat dog" file.txt

--- Failure 3 ---
Instruction: Display statistics for network interface eth0
Input: [WINDOWS]
Expected: netstat -e
Got: netstat -e eneth0

--- Failure 4 ---
Instruction: Check SELinux status for macOS
Input: 
Expected: echo N/A (SIP is different)
Got: echo N/A is common on Linux.

--- Failure 5 ---
Instruction: Save permission settings to a file using Linux
Input: 
Expected: getfacl -R . > permissions.bak
Got: getfacl > permissions.bak
